In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import specam
from specam.constants import abs0
from specam.data import SpectralDataGenerated

%matplotlib inline

In [ ]:
camera_name = 'specim_fx17'
camera_specs = {
    'kind': 'linspace',
    'start': 900e-9,
    'stop': 1700e-9,
    'num': 224,
    'signal_noise_ratio': 1000,
    'colour': 'red',
}
camera = specam.Camera.create(camera_name, **camera_specs)


In [ ]:
# Generate test thermal image
# resolution = (512, 640)
# resolution = (256, 320)
resolution = (128, 160)
grid = np.meshgrid(*[np.linspace(0, 1, x) for x in resolution], indexing="ij")
T_true = (np.sin(20*grid[0]) * grid[1] * grid[0] + 1) / 2
T_true = T_true * 3000 + 10 + abs0
T_true = T_true.flatten()

spectral_data = SpectralDataGenerated.create(
    camera.props["lam_vals"],
    T_true,
    specam.models.intensity_func,
    specam.models.intensity_func_log,
    {
        'C': 0.05, 'D': 0.9, 
        'lam_0': camera.props["lam_vals"][0],
        'lam_inf': camera.props["lam_vals"][-1],
    },
    signal_noise_ratio = camera.props['signal_noise_ratio']
)

In [ ]:
results = specam.fit_data('scipy', spectral_data)

In [ ]:
# init_vals = np.vstack([results[x] for x in ['T', 'C', 'D']]).T
# results = specam.fit_data('scipy', spectral_data, init_vals=init_vals)

In [ ]:
T_true = spectral_data['T']
T_pred = results['T']
abs_error = T_pred - T_true
rel_error = abs_error / T_true
plot_error = rel_error * 100

fig, axes = plt.subplots(3, 1, figsize=(5, 8))

ax = axes[0]
ax.set_title('True temperature')
im = ax.imshow(T_true.reshape(resolution))
fig.colorbar(im, ax=ax, label='Temperature (K)')
ax.axis('off')

ax = axes[1]
ax.set_title('Predicted temperature')
im = ax.imshow(T_pred.reshape(resolution))
fig.colorbar(im, ax=ax, label='Temperature (K)')
ax.axis('off')

ax = axes[2]
ax.set_title('Prediction error')
absmax = np.max(np.abs(plot_error))
im =ax.imshow(
    plot_error.reshape(resolution), 
    vmin=-absmax, vmax=absmax, cmap='seismic'
)
fig.colorbar(im, ax=ax, label='Relative error (%)')
ax.axis('off')

plt.savefig('spectral_image_test.pdf')

In [ ]:
T_std_dev = np.sqrt(results['covar'][:, 0, 0])

plt.figure(figsize=(4, 3))
plt.hist(T_std_dev, bins=20, range=(0, 5));
plt.xlabel('Fit standard deviation temperature (K)')
plt.ylabel('Frequency')

In [ ]:
T_pred = results['T']
T_std_dev = np.sqrt(results['covar'][:, 0, 0])

abs_error = T_pred - T_true
sigma_error = abs_error / T_std_dev

plt.figure(figsize=(4, 3))
plt.hist(sigma_error, bins=20, range=(-5, 5));
plt.xlabel('Error (fit standard deviation)')
plt.ylabel('Frequency')